In [ ]:
import imgTransformations
import utils_jupyter
import utils

import os

import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
dirname = os.path.abspath('')
folder = os.path.join(dirname, 'MEFDatabase/source image sequences/')
images = [] 

# Pass the images list to the interactive selector
utils_jupyter.interactive_image_selector(folder, images)

Definisci qua i metodi e inseriscili nel dizionario qua sotto, NON CAMBIARE I NOMI

In [ ]:
methods = {
    'Average Fusion': imgTransformations.average_fusion,
    'Mertens Fusion': imgTransformations.mertens_fusion,
    'Laplacian Pyramid': imgTransformations.laplacian_pyramid_fusion,
    'Exposure Fusion' : imgTransformations.exposure_fusion,
    'Exposure Compensation': imgTransformations.exposure_compensation_fusion,
    'Enhanced Exposure': imgTransformations.enhanced_exposure_fusion,
    'Domain Transform': imgTransformations.domain_transform_fusion,
    'Domain Transform - Homebrew': imgTransformations.domain_transform_fusion,
    'Wavelet Fusion': imgTransformations.wavelet_fusion
}

In [ ]:
utils_jupyter.showcase_methods_tab(images, methods)

In [ ]:
def calculate_weight_maps(img):
    epsilon = 1e-6

    guidance = img.astype(np.float32) / 255.0

    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    contrast = np.abs(cv2.Laplacian(gray, cv2.CV_32F))

    saturation = np.std(img, axis=2)

    well_exposedness = np.exp(-0.5 * ((img - 0.5) / 0.2) ** 2)
    well_exposedness = np.prod(well_exposedness, axis=2)

    weight = (contrast + epsilon) * (saturation + epsilon) * (well_exposedness + epsilon)

    # Apply Homebrew Domain Transform filter
    smooth_weight_homebrew = imgTransformations.dt_filter_homebrew(guidance, weight, sigmaSpatial=60, sigmaColor=0.4, num_iterations=2)

    # Apply OpenCV's dtFilter
    smooth_weight_opencv = cv2.ximgproc.dtFilter(img, weight.astype(np.float32), sigmaSpatial=60, sigmaColor=0.4, mode=1)
    
    return weight, smooth_weight_homebrew, smooth_weight_opencv


utils_jupyter.showcase_weight_maps_tab(images, calculate_weight_maps)



In [ ]:
#comparison between homebrew and opencv dt filter
title, fused = imgTransformations.domain_transform_fusion(images, homebrew_dt=False)
title_hb, fused_hb = imgTransformations.domain_transform_fusion(images, homebrew_dt=True)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.imshow(fused)
plt.title(title)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(fused_hb)
plt.title(title_hb)
plt.axis('off')

In [ ]:
def calculate_entropy(image):
    """Calculates entropy for an RGB image by averaging entropy across channels."""
    entropy_values = []
    for channel in range(image.shape[-1]):  # Loop through R, G, B channels
        hist, _ = np.histogram(image[:, :, channel].ravel(), bins=256, range=(0, 256))
        hist = hist.astype(np.float32) / hist.sum()
        hist = hist[hist > 0]  # Remove zero probabilities
        entropy_values.append(-np.sum(hist * np.log2(hist)))
    return np.mean(entropy_values)  # Average entropy across channels

def spatial_frequency(image):
    """Calculates spatial frequency for an RGB image."""
    sf_values = []
    for channel in range(image.shape[-1]):  # Compute SF for each color channel
        rows, cols = image.shape[:2]
        row_freq = np.sqrt(np.sum(np.diff(image[:, :, channel], axis=0) ** 2) / (rows * cols))
        col_freq = np.sqrt(np.sum(np.diff(image[:, :, channel], axis=1) ** 2) / (rows * cols))
        sf_values.append(np.sqrt(row_freq**2 + col_freq**2))
    return np.mean(sf_values)  # Average SF across channels

In [ ]:
def calculate_metrics_comparison(img1, img2):
    """Calculate similarity metrics between two images"""
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Convert to grayscale for SSIM/PSNR
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    # Calculate SSIM
    ssim_value = ssim(gray1, gray2, data_range=255)
    
    # Calculate PSNR with error handling
    mse = np.mean((gray1 - gray2) ** 2)
    if mse == 0:
        psnr_value = np.inf
    else:
        with np.errstate(divide='ignore'):
            psnr_value = 20 * np.log10(255) - 10 * np.log10(mse)
    
    return ssim_value, psnr_value

utils_jupyter.compare_methods_and_metrics(spatial_frequency, calculate_entropy, calculate_metrics_comparison, images, methods)

In [ ]:
def compute_difference(image1, image2):
    """Compute absolute difference between two images."""
    return cv2.absdiff(image1, image2)

def show_heatmap(diff_image):
    """Display a heatmap of the differences."""
    diff_gray = cv2.cvtColor(diff_image, cv2.COLOR_RGB2GRAY)
    plt.imshow(diff_gray, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title("Difference Heatmap")
    plt.show()

utils_jupyter.compare_methods_display(images, compute_difference, show_heatmap, methods)